In [ ]:
import sys as _sys
import os

current_path = os.path.abspath(os.getcwd())

split = current_path.split("inverse_geometric_locomotion")
if len(split)<2:
    print("Please rename the repository 'inverse_geometric_locomotion'")
    raise ValueError
path_to_python_scripts = os.path.join(split[0], "inverse_geometric_locomotion/python/")
path_to_notifications = os.path.join(split[0], "inverse_geometric_locomotion/notebooks/notifications/")
path_to_settings = os.path.join(split[0], "inverse_geometric_locomotion/python/figures/")
path_to_cubic_splines = os.path.join(split[0], "inverse_geometric_locomotion/ext/torchcubicspline/")
path_to_output = os.path.join(split[0], "inverse_geometric_locomotion/output/")
path_to_data = path_to_output
path_to_save = os.path.join(path_to_output, "snake_broken_joint")

_sys.path.insert(0, path_to_python_scripts)
_sys.path.insert(0, path_to_settings)
_sys.path.insert(0, path_to_cubic_splines)

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import shutil
import torch

from physics_quantities_torch import vmap_energy_torch
from utils import vmap_euc_transform_torch, vmap_euc_transform_T_torch
from vis_utils import produce_video_from_path, print_json_data
from vis_utils_snake import plot_animated_snake

## Load data

This notebook can be used to read files generated from executing `python/experiments/snake_broken_joint.py`. The following list assumes you have executed the experiment script successfully with the flag `--trial_number=0`. 

As you run more experiments with different `trial_number`, you can add the output path to the list `exp_file_names` with the corresponding file locations. The naming convention for `.json` files is `*_opt_{trial_number}.json`, where `{trial_number}` always has two digits.

In [ ]:
exp_file_names = [
    "snake_broken_joint/snake_broken_joints_opt_00.json",
]

trial_numbers = [int(fn.split('_')[-1].split('.')[0]) for fn in exp_file_names]
exp_names = ["snake_broken_joints_{:02d}".format(tn) for tn in trial_numbers]

list_js_loads = []
for exp_file_name in exp_file_names:
    with open(os.path.join(path_to_data, exp_file_name)) as jsonFile:
        js_load = json.load(jsonFile)

    print(exp_file_name)
    print_json_data(js_load)
    list_js_loads.append(js_load)

print("\nLoaded {} experiments.".format(len(list_js_loads)))

## Plot energy dissipation across experiments

Inspect and compare the energy dissipation of the optimal gaits on the same plot for all the experiments listed in `exp_file_names`.

In [ ]:

list_energies = []
for js_load in list_js_loads:

    pos_ = torch.tensor(js_load['pos_'])
    pos = torch.tensor(js_load['pos'])
    n_points_snake = pos.shape[1]
    n_ts = pos.shape[0]
    g = torch.tensor(js_load['g'])
    gt = torch.tensor(js_load['optimization_settings']['gt'])
    params_opt = torch.tensor(js_load['optimization_settings']['params_opt'])
    rho = js_load['optimization_settings']['rho']
    eps = js_load['optimization_settings']['eps']
    
    def fun_anisotropy_dir(x):
        tangents = torch.zeros_like(x)
        tangents[..., 0, :] = x[..., 1, :] - x[..., 0, :]
        tangents[..., 1:-1, :] = x[..., 2:, :] - x[..., :-2, :]
        tangents[..., -1, :] = x[..., -1, :] - x[..., -2, :]
        tangents = tangents / torch.linalg.norm(tangents, dim=-1, keepdims=True)
        return tangents

    tangents_ = fun_anisotropy_dir(pos_)
    pos = vmap_euc_transform_torch(g, pos_)
    tangents = vmap_euc_transform_T_torch(g, tangents_)
    edge_lengths = np.linalg.norm(pos_[:, 1:] - pos_[:, :-1], axis=-1)
    voronoi_cell_lengths = np.zeros(shape=(n_ts, n_points_snake))
    voronoi_cell_lengths[:, :-1] = edge_lengths / 2.0
    voronoi_cell_lengths[:, 1:] = edge_lengths / 2.0
    masses = rho * torch.tensor(np.tile(voronoi_cell_lengths[0], reps=(n_ts, 1)))
    a_weights = torch.ones(size=(n_ts, n_points_snake))
    b_weights = (eps - 1.0) * torch.ones(size=(n_ts, n_points_snake))

    energies = vmap_energy_torch(
        pos[:-1], pos[1:], tangents[:-1], tangents[1:], 
        masses[:-1], masses[1:], a_weights[:-1], a_weights[1:], 
        b_weights[:-1], b_weights[1:],
    )
    list_energies.append(energies)

fig = plt.figure(figsize=(6, 3))
gs = fig.add_gridspec(1, 1)
ax_tmp = fig.add_subplot(gs[0, 0])
for energies in list_energies:
    ax_tmp.plot(energies, lw=3.0, zorder=0)
ax_tmp.set_title("Dissipation energy")
plt.show()

## Produce an animation of the gait

Animate the previous plot and save it in the same folder in which you saved the output `.json` files.

You can list all the experiments you would like to generate the trajectories from by filling out `exp_id_list` with all the corresponding experiments indices in `exp_file_names`. By default, it will only generate the animation for the first experiment in the list.

In [ ]:
exp_id_list = [0] # Update this list with the indices of the experiments you want to animate

path_to_images_anim = os.path.join(path_to_save, "images_registered")
arrow_params = {
    "length": 0.05,
    "width": 0.02,
}

for exp_id in exp_id_list:
    print("Producing animation for experiment: {}".format(exp_file_names[exp_id]))

    js_load = list_js_loads[exp_id]
    exp_name = exp_names[exp_id]
    pos_plot = np.array(js_load['pos'])
    g_plot = np.array(js_load['g'])
    broken_joint_ids = js_load['optimization_settings']['broken_joint_ids']

    # clear existing images
    if os.path.exists(path_to_images_anim):
        shutil.rmtree(path_to_images_anim)
    os.makedirs(path_to_images_anim)

    plot_animated_snake(
        pos_plot, path_to_images_anim,
        g=g_plot, gt=gt, gcp=None,
        broken_joint_ids=broken_joint_ids, obstacle=None,
        exponent=1.0, xy_lim=None, 
        show_orientation=False, show_snake_trail=False, 
        show_g_trail=True, show_g_start=True, show_joints=True,
        arrow_params=arrow_params,
    )

    fn_pattern = os.path.join(path_to_images_anim, "step_%05d.png")
    produce_video_from_path(
        fn_pattern, path_to_save, 
        "{}_pos.mp4".format(exp_name), overwrite_anim=True, transparent=False
    )